## Optimize Model

## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN'])
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [7]:
# Look at NAME value counts for binning
name_counts = application_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [11]:
#  How many name counts are greater than 1 (Applicant applied more than once)
name_counts[name_counts>1]

PARENT BOOSTER USA INC                                            1260
TOPS CLUB INC                                                      765
UNITED STATES BOWLING CONGRESS INC                                 700
WASHINGTON STATE UNIVERSITY                                        492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                    408
                                                                  ... 
VETERANS OF FOREIGH WARS OF THE UNITED STATES DEPT OF COLORADO       2
ETA PHI BETA SORORITY INC                                            2
POINT MAN INTERNATIONAL MINISTRIES                                   2
MULTI COMMUNITY DIVERSIFIED SERVICES INC                             2
VETERANS OF FOREIGN WARS AUXILIARY                                   2
Name: NAME, Length: 792, dtype: int64

In [13]:
# Determine which values to replace if name counts is 1.
names_to_replace = list(name_counts[name_counts == 1].index)

# Replace in dataframe
for nm in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(nm,"Other")
    
# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                                                 18776
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
                                                                      ...  
WASHINGTON EXPLORER SEARCH AND RESCUE                                     2
INTERNATIONAL ALLIANCE THEATRICAL STAGE EMPLOYEES & MOVING PICTURE        2
BIRTH NETWORK                                                             2
NATIONAL ASSOCIATON OF CORPORATE DIRECTORS                                2
NATIONAL SOCIETY COLONIAL DAMES XVII CENTURY                              2
Name: NAME, Length: 793, dtype: int64

In [14]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [18]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [20]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_wo_1 = classification_counts[classification_counts > 1]
classification_counts_wo_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [21]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [22]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_dummies = pd.get_dummies(application_df)
categorical_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_4-H CLUBS & AFFILIATED 4-H ORGANIZATIONS,NAME_AACE INTERNATIONAL,NAME_ACADEMY OF GENERAL DENTISTRY,NAME_ACADEMY OF MANAGED CARE PHARMACY,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTIVE 20-30 UNITED STATES AND CANADA,NAME_ACTS MINISTRY,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [23]:
# Split our preprocessed data into our features and target arrays
X = categorical_dummies.drop('IS_SUCCESSFUL', axis=1).values
y = categorical_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
#Model 2
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden2_nodes_layer1 = 10
hidden2_nodes_layer2 = 8
hidden2_nodes_layer3 = 8

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer1, input_dim = number_input_features, activation = "relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer2, activation = "relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer3, activation = "relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                8360      
                                                                 
 dense_5 (Dense)             (None, 8)                 88        
                                                                 
 dense_6 (Dense)             (None, 8)                 72        
                                                                 
 dense_7 (Dense)             (None, 1)                 9         
                                                                 
Total params: 8529 (33.32 KB)
Trainable params: 8529 (33.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
fit_model2 = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5114 - accuracy: 0.7568
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4331 - accuracy: 0.7939
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4198 - accuracy: 0.8010
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4149 - accuracy: 0.8029
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4114 - accuracy: 0.8049
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4093 - accuracy: 0.8049
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4088 - accuracy: 0.8055
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4072 - accuracy: 0.8060
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4065 - accuracy: 0.8059
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4057 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.3914 - accuracy: 0.8154
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3914 - accuracy: 0.8146
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3913 - accuracy: 0.8156
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.3912 - accuracy: 0.8159
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3915 - accuracy: 0.8151
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3911 - accuracy: 0.8148
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3908 - accuracy: 0.8165
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3910 - accuracy: 0.8155
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3907 - accuracy: 0.8156
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3902 - accuracy: 

In [34]:
# Evaluate the model 2 using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4724 - accuracy: 0.7941 - 1s/epoch - 4ms/step
Loss: 0.4724194407463074, Accuracy: 0.7940524816513062


In [36]:
# Export our model to HDF5 file
nn2.save('h5_files/AlphabetSoupCharity_Optimization.h5')

## Conclusion

By optimizing the model we are able to increase the accuracy to around 79%, which means we are able to correclty classify each of the data points in the test data 79% of the time.
In order to do this we instead used the NAME column (not eliminate it) and filtered the dataframe to those applicants that had applied more than once. A third hidden layer was also added in order to help increase the accuracy by increasing the depth of the model since our input data seems to have complex dependencies and hierarchies. 

